#### Necessary Imports 

In [25]:
import os
from openai import OpenAI
import getpass


openai_key = getpass.getpass()
client = OpenAI(api_key = openai_key)
os.environ["OPENAI_API_KEY"] = getpass.getpass()

········
········


####  Chat API by OpenAI



In [26]:
llm_model = 'gpt-4o'
system_content = 'You are a friendly assistant who will help the user with their queries'
user_content = 'What is water cycle ?'

####  Chechking the connection


In [27]:
response = client.chat.completions.create(
    model= llm_model,
    messages=[
    {"role": "system", "content": system_content},
    {"role": "user", "content": user_content}]
)


print(response.choices[0].message.content)

The water cycle, also known as the hydrological cycle, describes the continuous movement of water on, above, and below the surface of the Earth. This cycle is crucial for distributing water across the planet and involves several key processes:

1. **Evaporation**: Water from oceans, rivers, lakes, and other bodies of water heats up and turns into water vapor, rising into the atmosphere.

2. **Transpiration**: Water is also released from plants and soil into the atmosphere as water vapor through a process called transpiration.

3. **Condensation**: As water vapor rises and cools, it turns back into liquid water droplets, forming clouds in the atmosphere.

4. **Precipitation**: When water droplets in clouds combine and become heavy enough, they fall back to the Earth's surface as precipitation (rain, snow, sleet, or hail).

5. **Infiltration**: Some of the water that precipitates onto the ground soaks into the soil, becoming groundwater.

6. **Runoff**: Water that does not infiltrate the

####  ChatCompletion with translation task


In [28]:
complaint_recieved = "Mera order abhi tak nahi aaya. Mai Kya karun"
desired_tone = "friendly and helpful"

In [29]:
prompt = f"""Respond to the complaint recieved in English \
that is delimited by single quotes
with a tone that is {desired_tone}.
text: '{complaint_recieved}'
"""

print(prompt)

Respond to the complaint recieved in English that is delimited by single quotes
with a tone that is friendly and helpful.
text: 'Mera order abhi tak nahi aaya. Mai Kya karun'



####  Responding to the complaint 

In [30]:
response =  client.chat.completions.create(
    model= llm_model,
    messages=[
    {"role": "system", "content": system_content},
    {"role": "user", "content": prompt}]
)

In [31]:
print(response.choices[0].message.content)

Dear Customer,

Thank you for reaching out to us. I understand your concern about your order not arriving yet, and I'm here to help you.

Could you please provide me with your order number or any other relevant details? This will help me to quickly locate your order status and update you on its progress.

Your patience and understanding are greatly appreciated. We're committed to ensuring that you receive your order as soon as possible.

Looking forward to assisting you further!

Best regards,
[Your Name]


####  LangChain


  In this section, you will establishing the connection with OpenAI using LangChain

In [33]:
from langchain_openai import ChatOpenAI

In [34]:

chat_call = ChatOpenAI(temperature=0.0, model=llm_model)
chat_call

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10bcfce50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10cbfe590>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

#### Prompt template demonstration

In [35]:
template_string = """Respond to the complaint received in English \
that is delimited by single quotes
with a tone that is {tone}.
text: '{complaint}'
"""

In [36]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)


In [37]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['complaint', 'tone'], template="Respond to the complaint received in English that is delimited by single quotes\nwith a tone that is {tone}.\ntext: '{complaint}'\n")

In [38]:
prompt_template.messages[0].prompt.input_variables

['complaint', 'tone']

In [39]:
user_request = prompt_template.format_messages(
                    tone=desired_tone,
                    complaint=complaint_recieved)

In [40]:
print((user_request))

[HumanMessage(content="Respond to the complaint received in English that is delimited by single quotes\nwith a tone that is friendly and helpful.\ntext: 'Mera order abhi tak nahi aaya. Mai Kya karun'\n")]


In [45]:
customer_response = chat_call.invoke(user_request)
print(customer_response.content)

Dear Customer,

Thank you for reaching out to us. I'm sorry to hear that your order hasn't arrived yet. Let's get this sorted out for you as quickly as possible.

Could you please provide me with your order number and any other relevant details? This will help me track your order and give you an update on its status.

Thank you for your patience, and I look forward to resolving this for you soon!

Best regards,
[Your Name]


#### Output Parsers Demonstration

In [50]:
product_description = """\
The amazing Hercules sofa is a classic 3-seater sofa in\
velvet texture. Easy to clean with modern style, the sofa\
comes in five colours - blue, yellow, olive, grey and white\
Uplift the vibe of your living room in just 999 euros with the new Hercules. 
"""

In [51]:
description_template = """\
For the following text, extract the information in the format described below:

Brand:  If the brand name is mentioned, extract its value else the value is 'unknown'

Item:  If the furniture item is mentioned, extract its value else the value is 'unknown'

Size: If the dimensions are mentioned then extract those \
      if the seating capacity is mentioned extract that\
      otherwise the value is 'unknown' 

Texture: If the texture is mentioned, extract its value else the value is 'unknown'

Colours: If any colour is mentioned, extract the colour names else the value is 'unknown'

Price: If the price is mentioned, extract its value along with the currency else the value is 'unkonwn'

Format the final output as a JSON with the following keys:

'Brand'
'Item' 
'Size'
'Texture'
'Colours'
'Price'

text: {text}
"""

In [52]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(description_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template="For the following text, extract the information in the format described below:\n\nBrand:  If the brand name is mentioned, extract its value else the value is 'unknown'\n\nItem:  If the furniture item is mentioned, extract its value else the value is 'unknown'\n\nSize: If the dimensions are mentioned then extract those       if the seating capacity is mentioned extract that      otherwise the value is 'unkown' \n\nTexture: If the texture is mentioned, extract its value else the value is 'unknown'\n\nColours: If any colour is mentioned, extract the colour names else the value is 'unknown'\n\nPrice: If the price is mentioned, extract its value along with the currecy else the value is 'unkown'\n\nFormat the final output as a JSON with the following keys:\n\n'Brand'\n'Item' \n'Size'\n'Texture'\n'Colours'\n'Price'\n\ntext: {text}\n"))]


In [56]:
messages = prompt_template.format_messages(text=product_description)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response)

content='```json\n{\n  "Brand": "Hercules",\n  "Item": "sofa",\n  "Size": "3-seater",\n  "Texture": "velvet",\n  "Colours": ["blue", "yellow", "olive", "grey", "white"],\n  "Price": "999 euros"\n}\n```' response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 246, 'total_tokens': 313}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'stop', 'logprobs': None} id='run-595f1623-fa7a-4203-bef3-4fdb34ffe0dd-0' usage_metadata={'input_tokens': 246, 'output_tokens': 67, 'total_tokens': 313}


In [57]:
print(type(response.content))

<class 'str'>


In [60]:
print((response.content))

```json
{
  "Brand": "Hercules",
  "Item": "sofa",
  "Size": "3-seater",
  "Texture": "velvet",
  "Colours": ["blue", "yellow", "olive", "grey", "white"],
  "Price": "999 euros"
}
```


In [59]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('Brand')

AttributeError: 'str' object has no attribute 'get'

#### Parsing the  output  into a Python dictionary

In [61]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [75]:
brand_schema = ResponseSchema(name="Brand",
                             description="If the brand name is mentioned, \
                             extract its value else the value is 'unknown'")
item_schema = ResponseSchema(name="Item",
                             description="If the furniture item is mentioned,\
                                          extract its value else the value is 'unknown'")
size_schema = ResponseSchema(name="Size",
                             description="If the dimensions are mentioned then extract those \
                                          if the seating capacity is mentioned extract that\
                                          otherwise the value is 'unknown' ")

texture_schema = ResponseSchema(name="Texture",
                                description="If the texture is mentioned, extract its value \
                                             else the value is 'unknown'")

colours_schema = ResponseSchema(name="Colours",
                                description="If any colour is mentioned, extract the colour names \
                                else the value is 'unknown''")

price_schema = ResponseSchema(name="Price",
                              description="If the price is mentioned, extract its value along with the currency\
                                           else the value is 'unknown' ")

response_schema = [brand_schema, 
                    item_schema,
                    size_schema,
                    texture_schema,
                    colours_schema,
                    price_schema]

In [77]:
op_parser = StructuredOutputParser.from_response_schemas(response_schema)
get_instructions = op_parser.get_format_instructions()
print(get_instructions)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Brand": string  // If the brand name is mentioned,                              extract its value else the value is 'unknown'
	"Item": string  // If the furniture item is mentioned,                                          extract its value else the value is 'unknown'
	"Size": string  // If the dimensions are mentioned then extract those                                           if the seating capacity is mentioned extract that                                          otherwise the value is 'unknown' 
	"Texture": string  // If the texture is mentioned, extract its value                                              else the value is 'unknown'
	"Colours": string  // If any colour is mentioned, extract the colour names                                 else the value is 'unknown''
	"Price": string  // If the price is mentioned, extract its valu

In [78]:
prompt_template = ChatPromptTemplate.from_template(template=description_template)

format_message = prompt_template.format_messages(text=product_description, 
                                  format_instructions=get_instructions)

In [80]:
print(format_message[0].content)

For the following text, extract the information in the format described below:

Brand:  If the brand name is mentioned, extract its value else the value is 'unknown'

Item:  If the furniture item is mentioned, extract its value else the value is 'unknown'

Size: If the dimensions are mentioned then extract those       if the seating capacity is mentioned extract that      otherwise the value is 'unkown' 

Texture: If the texture is mentioned, extract its value else the value is 'unknown'

Colours: If any colour is mentioned, extract the colour names else the value is 'unknown'

Price: If the price is mentioned, extract its value along with the currecy else the value is 'unkown'

Format the final output as a JSON with the following keys:

'Brand'
'Item' 
'Size'
'Texture'
'Colours'
'Price'

text: The amazing Hercules sofa is a classic 3-seater sofa invelvet texture. Easy to clean with modern style, the sofacomes in five colours - blue, yellow, olive, grey and whiteUplift the vibe of your

In [81]:
response = chat(format_message)

In [82]:
print(response.content)

```json
{
  "Brand": "Hercules",
  "Item": "sofa",
  "Size": "3-seater",
  "Texture": "velvet",
  "Colours": ["blue", "yellow", "olive", "grey", "white"],
  "Price": "999 euros"
}
```


In [88]:
output = op_parser.parse(response.content)

In [89]:
output

{'Brand': 'Hercules',
 'Item': 'sofa',
 'Size': '3-seater',
 'Texture': 'velvet',
 'Colours': ['blue', 'yellow', 'olive', 'grey', 'white'],
 'Price': '999 euros'}

In [90]:
output['Brand']

'Hercules'